# Exercise - SVM

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. This is a binary classification task: predict whether a loan will be bad or not (1=Yes, 0=No). This is an important task for banks to prevent bad loans from being issued.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

## Goal

Use the **loan.csv** data set and build a model to predict **BAD**. 

Since you have a relatively small data set, I recommend using cross-validation to evaluate your accuracy.

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

loan = pd.read_csv("loan.csv")
loan.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,0,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,1,50000,220528.0,300900.0,HomeImp,Self,5.0,0.0,0.0,0.000000,0.0,2.0,NaN
3,1,22400,51470.0,68139.0,DebtCon,Mgr,9.0,0.0,0.0,31.168696,2.0,8.0,37.952180
4,0,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(loan, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Separate the target variable 

In [5]:
train_target = train['BAD']
test_target = test['BAD']

train_inputs = train.drop(['BAD'], axis=1)
test_inputs = test.drop(['BAD'], axis=1)

##  Identify the numeric, binary, and categorical columns

In [6]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [7]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [8]:
categorical_columns

['REASON', 'JOB']

# Pipeline

In [9]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [10]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [11]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [12]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[-0.31412013, -1.30301181, -0.86148829, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.45733454,  0.7398414 ,  0.58636192, ...,  1.        ,
         0.        ,  0.        ],
       [-1.10330939,  0.2001631 ,  0.18146318, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.21657988, -0.83000156, -0.82081329, ...,  0.        ,
         0.        ,  0.        ],
       [-0.46486414,  1.79196675,  1.36974799, ...,  0.        ,
         0.        ,  0.        ],
       [-0.31412013, -0.08740643, -0.21782887, ...,  0.        ,
         0.        ,  0.        ]])

In [13]:
train_x.shape

(1750, 20)

# Tranform: transform() for TEST

In [14]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.06717356,  0.36706438,  0.32127798, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.32432512,  0.57631513,  0.42769944, ...,  0.        ,
         1.        ,  0.        ],
       [-0.33185472,  0.41209537,  0.12227549, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.8993616 , -0.51071616, -0.32326299, ...,  0.        ,
         0.        ,  0.        ],
       [-0.5446698 , -0.82706576, -0.8366813 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.5446698 , -0.06422056, -0.11380525, ...,  0.        ,
         0.        ,  0.        ]])

In [15]:
test_x.shape

(750, 20)

# Calculate the Baseline

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
#First find the average value of the target

mean_value = np.mean(train_target)

mean_value

0.3965714285714286

In [18]:
# Predict all values as the mean

baseline_pred = np.repeat(mean_value, len(test_target))

baseline_pred


array([0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657143,
       0.39657143, 0.39657143, 0.39657143, 0.39657143, 0.39657

In [19]:
test_target

1447    0
1114    0
1064    0
2287    1
1537    1
       ..
427     1
105     0
2044    1
121     1
821     1
Name: BAD, Length: 750, dtype: int64

In [20]:
baseline_mse = mean_squared_error(test_target, baseline_pred)

baseline_rmse = np.sqrt(baseline_mse)

print('Baseline RMSE: {}' .format(baseline_rmse))

Baseline RMSE: 0.49467213081676914


## BASELINE CALCULATION (MANUALLY)

In [21]:
# Find majority class_manually
train_target.value_counts()

0    1056
1     694
Name: BAD, dtype: int64

In [22]:
# Find percentage _ manually
train_target.value_counts()/len(train_target)

0    0.603429
1    0.396571
Name: BAD, dtype: float64

# Train an SVM model with linear kernel

In [23]:
from sklearn.svm import SVC
 
lin_svm = SVC(kernel="linear")

lin_svm.fit(train_x, train_target)

SVC(kernel='linear')

In [24]:
from sklearn.metrics import accuracy_score

### Calculate the accuracy

In [25]:
#Predict the train values
train_y_pred = lin_svm.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_y_pred)

0.7525714285714286

In [32]:
test_x.shape

(750, 20)

In [33]:
#Predict the test values
test_y_pred = lin_svm.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_y_pred)

0.7266666666666667

# Train an SVM model with poly kernel

In [34]:
from sklearn.svm import SVC

# You need to enter a value for gamma. Remember, gamma controls the shape of the bell curve for rbf
# You can also set it is as gamma='scale'. This will be the default option in future releases

pol_svm = SVC(kernel="poly", degree=3, coef0=1, C=10)

pol_svm.fit(train_x, train_target)

SVC(C=10, coef0=1, kernel='poly')

## Calculate the accuracy

In [36]:
#Predict the train values
train_y_pred = pol_svm.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_y_pred)

0.8965714285714286

In [37]:
#Predict the test values
test_y_pred = pol_svm.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_y_pred)

0.8173333333333334

# Train an SVM model with rbf kernel

In [40]:
rbf_svm = SVC(kernel="rbf", C=10, gamma='scale')

rbf_svm.fit(train_x, train_target)

SVC(C=10)

### Calculate the accuracy

In [42]:
#Predict the train values
train_y_pred = rbf_svm.predict(train_x)

#Train accuracy
accuracy_score(train_target, train_y_pred)

0.9177142857142857

In [43]:
#Predict the test values
test_y_pred = rbf_svm.predict(test_x)

#Test accuracy
accuracy_score(test_target, test_y_pred)

0.8453333333333334